# Compare classification methods for identifying org. science perspectives in JSTOR articles
## Using balanced samples from hand-labeled set of articles

@author: Jaren Haber, PhD<br>
@coauthors: Prof. Heather Haveman, UC Berkeley; Yoon Sung Hong, Wayfair<br>
@contact: Jaren.Haber@georgetown.edu<br>
@project: Computational Literature Review of Organizational Scholarship<br>
@date: December 2020

'''
Trains classifiers to predict whether an article is about a given perspective in org. science. To train the classifiers, uses preliminary labeled articles, broken down as follows: 
Cultural: 105 yes, 209 no
Relational: 92 yes, 230 no
Demographic: 77 yes, 249 no
Compares f1_weighted scores of four model structures using 10-Fold Cross Validation: Logistic regression, SVM, Naive Bayes, and Decision Tree. Oversamples training data to .7 (7:10 minority:majority class).
'''

## Initialize

In [6]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /opt/conda/lib/python3.7/site-packages (0.7.0)


In [9]:
######################################################
# Import libraries
######################################################

import pandas as pd
import numpy as np
import re
from collections import Counter
from datetime import date
import os

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')

stemmer = WordNetLemmatizer()

from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

import joblib
import csv

from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, KFold

# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler

import sys; sys.path.insert(0, "../preprocess/") # For loading functions from files in other directory
from quickpickle import quickpickle_dump, quickpickle_load # custom scripts for quick saving & loading to pickle format

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
######################################################
# Define filepaths
######################################################

thisday = date.today().strftime("%m%d%y")

cwd = os.getcwd()
root = str.replace(cwd, 'classification/modeling', '')

# Directory for prepared data and trained models: save files here
data_fp = root + 'classification/data/'
model_fp = root + 'classification/models/'

# Current article lists
article_list_fp = data_fp + 'filtered_length_index.csv' # Filtered index of research articles
article_paths_fp = data_fp + 'filtered_length_article_paths.csv' # List of article file paths

# Preprocessed training data
cult_labeled_fp = data_fp + 'training_cultural_preprocessed_121620.pkl'
relt_labeled_fp = data_fp + 'training_relational_preprocessed_121620.pkl'
demog_labeled_fp = data_fp + 'training_demographic_preprocessed_121620.pkl'

# Model filepaths
cult_model_fp = model_fp + f'classifier_cult_{str(thisday)}.joblib'
relt_model_fp = model_fp + f'classifier_relt_{str(thisday)}.joblib'
demog_model_fp = model_fp + f'classifier_demog_{str(thisday)}.joblib'

# Vectorizers trained on hand-coded data (use to limit vocab of input texts)
cult_vec_fp = model_fp + 'vectorizer_cult_121620.joblib'
relt_vec_fp = model_fp + 'vectorizer_relt_121620.joblib'
demog_vec_fp = model_fp + 'vectorizer_demog_121620.joblib'

## Load & inspect data

In [11]:
cult_df = quickpickle_load(cult_labeled_fp)
relt_df = quickpickle_load(relt_labeled_fp)
demog_df = quickpickle_load(demog_labeled_fp)

cult_df.head(10)

,text,cultural_score
0,"[[journal, of, managerial, issues, vol, xxiii,...",1.0
1,"[[organization, ht, icna, vol, no, may, june, ...",1.0
2,"[[from, fiefs, to, clans, and, network, capita...",1.0
3,"[[the, collective, strategy, framework, an, ap...",1.0
4,"[[manag, int, rev, doi, sl, research, article,...",1.0
5,"[[int, studies, ofmgt, amp, org, vol, no, pp, ...",1.0
6,"[[linking, organizational, values, to, relatio...",1.0
7,"[[journal, of, organizational, behavior, organ...",1.0
8,"[[®, academy, oí, management, learning, amp, e...",1.0
9,"[[strategie, management, journal, strat, mgmt,...",1.0


In [12]:
# Check score distribution across classes
print(cult_df.groupby('cultural_score').size())
print()
print(relt_df.groupby('relational_score').size())
print()
print(demog_df.groupby('demographic_score').size())

cultural_score
0.0    209
0.5     12
1.0    133
dtype: int64

relational_score
0.0    229
0.5      7
1.0    114
dtype: int64

demographic_score
0.0    248
0.5      5
1.0    101
dtype: int64


In [13]:
# Drop unsure cases: where X_score = 0.5
drop_unsure = True

if drop_unsure:
    cult_df_yes = cult_df[cult_df['cultural_score'] == 1.0]
    cult_df_no = cult_df[cult_df['cultural_score'] == 0.0]
    cult_df = pd.concat([cult_df_yes, cult_df_no])
    
    relt_df_yes = relt_df[relt_df['relational_score'] == 1.0]
    relt_df_no = relt_df[relt_df['relational_score'] == 0.0]
    relt_df = pd.concat([relt_df_yes, relt_df_no])
    
    demog_df_yes = demog_df[demog_df['demographic_score'] == 1.0]
    demog_df_no = demog_df[demog_df['demographic_score'] == 0.0]
    demog_df = pd.concat([demog_df_yes, demog_df_no])

### Check vocab size and frequent words

In [14]:
def collect_article_tokens(article, return_string=False):
    '''
    Collects words from already-tokenized sentences representing each article.
    
    Args:
        article: list of lists of words (each list is a sentence)
        return_string: whether to return single, long string representing article
    Returns:
        tokens: string if return_string, else list of tokens
    '''
    
    tokens = [] # initialize
    
    if return_string:
        for sent in article:
            sent = ' '.join(sent) # make sentence into a string
            tokens.append(sent) # add sentence to list of sentences
        tokens = ' '.join(tokens) # join sentences into string
        return tokens # return string
    
    else:
        for sent in article:
            tokens += [word for word in sent] # add each word to list of tokens
        return tokens # return list of tokens

# For capturing word frequencies, add all words from each article to single, shared list (can't use this to create models)
cult_tokens = []; cult_df['text'].apply(lambda article: cult_tokens.extend([word for word in collect_article_tokens(article)]))
relt_tokens = []; relt_df['text'].apply(lambda article: relt_tokens.extend([word for word in collect_article_tokens(article)]))
demog_tokens = []; demog_df['text'].apply(lambda article: demog_tokens.extend([word for word in collect_article_tokens(article)]))
print()

In [15]:
# Look at size of vocabulary and most frequent words
tokens = (cult_tokens + relt_tokens) + demog_tokens
print('Vocab size:', len(set(tokens)))
print()

# Check out most frequent words in labeled texts
freq = Counter(tokens)
print('20 most frequent words in labeled articles:')
freq.most_common(20)

Vocab size: 80217

20 most frequent words in labeled articles:


[('the', 596822),
 ('of', 428762),
 ('and', 330459),
 ('in', 232360),
 ('to', 218627),
 ('that', 117768),
 ('is', 103616),
 ('for', 94102),
 ('as', 76669),
 ('on', 63680),
 ('are', 63279),
 ('with', 59141),
 ('by', 57364),
 ('this', 53800),
 ('be', 47394),
 ('oasis', 46263),
 ('or', 42620),
 ('it', 40088),
 ('entry', 39972),
 ('from', 39891)]

### Check frequent sentences (to improve cleaning)

In [16]:
# Add sentences from each article to empty list:
cult_sents = []; cult_df['text'].apply(
    lambda article: cult_sents.extend(
        [' '.join([word for word in sent]) for sent in article]))
relt_sents = []; relt_df['text'].apply(
    lambda article: relt_sents.extend(
        [' '.join([word for word in sent]) for sent in article]))
demog_sents = []; demog_df['text'].apply(
    lambda article: demog_sents.extend(
        [' '.join([word for word in sent]) for sent in article]))

sents = (cult_sents + relt_sents) + demog_sents
print('Number of sentences:', len(sents))
print()

# Check out most frequent sentences in labeled texts
freq = Counter(sents)
print('20 most frequent sentences in labeled articles:')
freq.most_common(20)

Number of sentences: 133028

20 most frequent sentences in labeled articles:


[('colsep', 12325),
 ('rowsep', 12319),
 ('oasis entry colname', 11914),
 ('char', 7881),
 ('align char', 7802),
 ('valign bottom oasis entry', 7243),
 ('oasis row', 6022),
 ('align center', 2775),
 ('valign bottom', 2595),
 ('oasis entry colname colsep rowsep align char char oasis entry', 1589),
 ('align left', 1573),
 ('oasis entry colname colsep rowsep align char char', 1250),
 ('oasis entry colsep rowsep valign bottom oasis entry', 1045),
 ('label label', 767),
 ('fn', 744),
 ('sec', 606),
 ('oasis colspec colnum colname colwidth pi', 573),
 ('sec id sc', 560),
 ('oasis entry colsep rowsep oasis entry', 531),
 ('fn id fn', 509)]

### Load and apply text vectorizers

In [17]:
# Collect articles: Add each article as single str to list of str:
cult_docs = [] # empty list
cult_df['text'].apply(
    lambda article: cult_docs.append(
        collect_article_tokens(
            article, 
            return_string=True)))

relt_docs = [] # empty list
relt_df['text'].apply(
    lambda article: relt_docs.append(
       collect_article_tokens(
            article, 
            return_string=True)))

demog_docs = [] # empty list
demog_df['text'].apply(
    lambda article: demog_docs.append(
        collect_article_tokens(
            article, 
            return_string=True)))

print() # skip weird output

In [18]:
# Define stopwords used by JSTOR
jstor_stopwords = set(["a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", "into", "is", "it", "no", "not", "of", "on", "or", "such", "that", "the", "their", "then", "there", "these", "they", "this", "to", "was", "will", "with"])

# Uses TFIDF weighted DTM because results in better classifier accuracy than unweighted
cult_vectorizer = joblib.load(cult_vec_fp, "r+")
X_cult = cult_vectorizer.transform(cult_docs)
print('Number of features in cultural vectorizer:', len(cult_vectorizer.get_feature_names()))
print(cult_vectorizer.get_feature_names()[::1000]) # get every 1000th word
print()

relt_vectorizer = joblib.load(relt_vec_fp, "r+")
X_relt = relt_vectorizer.transform(relt_docs)
print('Number of features in relational vectorizer:', len(relt_vectorizer.get_feature_names()))
print(relt_vectorizer.get_feature_names()[::1000]) # get every 1000th word
print()

demog_vectorizer = joblib.load(demog_vec_fp, "r+")
X_demog = demog_vectorizer.transform(demog_docs)
print('Number of features in demographic vectorizer:', len(demog_vectorizer.get_feature_names()))
print(demog_vectorizer.get_feature_names()[::1000]) # get every 1000th word

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  N_MAX_ELEMENTS_TO_SHOW = 30  # number of elements to show in sequences
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  N_MAX_ELEMENTS_TO_SHOW = 30  # number of elements to show in sequences


Number of features in cultural vectorizer: 66098
['aa', 'affects', 'analogize', 'argumentatively', 'aveni', 'beifuss', 'blustein', 'buchholtz', 'carron', 'childlessness', 'coinciding', 'confused', 'councillors', 'czabansku', 'demirguc', 'dijeffren', 'dogs', 'economets', 'endler', 'ethnidzed', 'fad', 'fiskarbankadministrasjon', 'frenzen', 'ger', 'greenwood', 'haskell', 'hobson', 'identified', 'indexes', 'interceded', 'itbs', 'kaleidoscope', 'kraiger', 'legalized', 'lockett', 'makrosoziologie', 'mcenough', 'millenarian', 'mot', 'ncjrs', 'noninstrumental', 'offaculty', 'oshpd', 'parsimonious', 'phase', 'poovanalingum', 'primatively', 'punctuate', 'readership', 'relabeled', 'review', 'ruddy', 'schneider', 'severed', 'site', 'sparrowe', 'stochas', 'superlative', 'taurus', 'thus', 'transitioning', 'ularco', 'unsafe', 'vessies', 'weekends', 'wrinkle', 'ïï']

Number of features in relational vectorizer: 64919
['aa', 'affordance', 'ancestors', 'arose', 'aydin', 'benefitted', 'bonacich', 'burkar

## Setup for modeling

In [19]:
######################################################
# Balance x_train, y_train
######################################################

def resample_data(X_train, Y_train, undersample = False, sampling_ratio = 1.0):
    """
    Args:
        X_train: X training data
        Y_train: Y training data
        undersample: boolean for over or undersampling
        sampling_ratio: ratio of minority to majority class
        
        archived/not used:
        sampling_strategy: strategy for resampled distribution
            if oversample: 'majority' makes minority = to majority
            if undersample: 'minority' makes majority = to minority
            
    Returns:
        X_balanced: predictors at balanced ratio
        Y_balanced: outcomes at balanced ratio
    """
    
    if undersample == True:
        undersample = RandomUnderSampler(sampling_strategy=sampling_ratio)
        X_balanced, Y_balanced = undersample.fit_resample(X_train, Y_train)
    else:
        oversample = RandomOverSampler(sampling_strategy=sampling_ratio)
        X_balanced, Y_balanced = oversample.fit_resample(X_train, Y_train)
    
    print(f'Y_train: {Counter(Y_train)}\nY_resample: {Counter(Y_balanced)}')
    
    return X_balanced, Y_balanced

In [20]:
######################################################
# k-fold cross validation for model evaluation
######################################################

# Define test options for k-fold CV
num_folds = 10 
seed = 3
scoring='f1_weighted' # set scoring metric (not used here)

def show_kfold_output(models, 
                      X, 
                      Y, 
                      num_folds = num_folds, 
                      random_state = seed, 
                      shuffle = True):
    '''
    Estimates the accuracy of different model algorithms, adds results to a results array and returns.
    Prints the accuracy results: averages and std.
    Uses cross_val_predict, which unlike cross_val_score cannot define scoring option/evaluation metric.
    
    Args:
        models: list of (name, model) tuples
        X: predictors
        Y: outcomes
        num_folds: Split data randomly into num_folds parts: (num_folds-1) for training, 1 for scoring
        random_state: seed
        shuffle: 
    
    Returns:
        results: list of model results
        names: list of model names (matches results)
        
    Source: 
        https://stackoverflow.com/questions/40057049/using-confusion-matrix-as-scoring-metric-in-cross-validation-in-scikit-learn
    '''
    
    results = []
    names = []
    
    for name, model in models:
        # Setup model options
        kfold = KFold(
            n_splits=num_folds, 
            random_state=seed, 
            shuffle=True)
        
        # Get kfold results
        cv_results = cross_val_predict(
            model, 
            X, 
            Y, 
            cv=kfold, 
            #scoring=scoring, 
            n_jobs=-1) # use all cores = faster
        
        # Add results and name of each algorithm to the model array
        results.append(cv_results)
        names.append(name)
        
        # Print results
        print(f'{name}:')
        print()
        print(f'Mean (std):\t {round(cv_results.mean(),4)} ({round(cv_results.std(),4)})')
        print(f'Accuracy:\t', {round(accuracy_score(Y_balanced, cv_results)), 4})
        print()
        print('Confusion matrix:\n', confusion_matrix(Y_balanced, cv_results))
        print()
        print('Report:\n', classification_report(Y_balanced, cv_results))
        print()
        
    # Return arrays
    return results, names

## Evaluate algorithms: Cultural perspective

In [21]:
######################################################
# Prepare training and validation data
######################################################

# Separate training and final validation data set. First remove class
# label from data (X). Setup target class (Y)
# Then make the validation set 10% of the entire
# set of labeled data (X_validate, Y_validate)

cult_df = cult_df[['text', 'cultural_score']]
print("Number of cases:", str(X_cult.shape[0]))

valueArray = cult_df.values
Y = valueArray[:,1]
Y = Y.astype('float')
print("Number of codes (should match):", str(len(Y)))

test_size = 0.2
seed = 3
X_train, X_validate, Y_train, Y_validate = train_test_split(
    X_cult, 
    Y, 
    test_size=test_size, 
    random_state=seed)

print(f'Y_train Distribution: {Counter(Y_train).most_common()}')

Number of cases: 342
Number of codes (should match): 342
Y_train Distribution: [(0.0, 165), (1.0, 108)]


In [28]:
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon

In [29]:
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()
data_ctx = ctx
model_ctx = ctx

In [30]:
X_train.shape

(273, 66098)

In [ ]:
num_hidden = 64
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

In [ ]:

# load the dataset
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

In [21]:
######################################################
# Oversample to desirable ratio
######################################################

# Use these settings here and below
sampling_ratio = 1.0 # ratio of minority to majority cases
undersample = False # whether to undersample or oversample

X_balanced, Y_balanced = resample_data(
    X_cult, 
    Y, 
    undersample=undersample, 
    sampling_ratio=sampling_ratio)

NameError: name 'RandomOverSampler' is not defined

In [17]:
######################################################
# Use different algorithms to build models
######################################################

models = []
models.append(('K-Nearest Neighbors (KNN)', KNeighborsClassifier()))
models.append(('Random Forest (RF)', RandomForestClassifier(random_state=seed)))
models.append(('Decision Tree (DT)', DecisionTreeClassifier(random_state=seed)))
models.append(('Multinomial Naive Bayes (MNB)', MultinomialNB()))
models.append(('Logistic Regression (LR)', LogisticRegression(random_state=seed)))
models.append(('Support Vector Machine (SVM)', SVC(gamma='auto')))

# Evaluate algorithms using 10-fold cross validation
results, names = show_kfold_output(models=models, 
                                   X=X_balanced, 
                                   Y=Y_balanced)

K-Nearest Neighbors (KNN):

Mean (std):	 0.6292 (0.483)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[137  72]
 [ 18 191]]

Report:
               precision    recall  f1-score   support

         0.0       0.88      0.66      0.75       209
         1.0       0.73      0.91      0.81       209

    accuracy                           0.78       418
   macro avg       0.81      0.78      0.78       418
weighted avg       0.81      0.78      0.78       418


Random Forest (RF):

Mean (std):	 0.4976 (0.5)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[190  19]
 [ 20 189]]

Report:
               precision    recall  f1-score   support

         0.0       0.90      0.91      0.91       209
         1.0       0.91      0.90      0.91       209

    accuracy                           0.91       418
   macro avg       0.91      0.91      0.91       418
weighted avg       0.91      0.91      0.91       418


Decision Tree (DT):

Mean (std):	 0.5359 (0.4987)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[171

In [18]:
######################################################
# Save best model
######################################################

rf_cult = RandomForestClassifier(random_state=seed).fit(X_balanced, Y_balanced)
joblib.dump(rf_cult, cult_model_fp)

['/home/jovyan/work/classification/models/classifier_cult_121620.joblib']

## Evaluate algorithms: Relational perspective

In [19]:
######################################################
# Prepare training and validation data
######################################################

# Separate training and final validation data set. First remove class
# label from data (X). Setup target class (Y)
# Then make the validation set 10% of the entire
# set of labeled data (X_validate, Y_validate)

relt_df = relt_df[['text', 'relational_score']]
print("Number of cases:", str(X_relt.shape[0]))

valueArray = relt_df.values
Y = valueArray[:,1]
Y = Y.astype('float')
print("Number of codes (should match):", str(len(Y)))

test_size = 0.2
seed = 3
X_train, X_validate, Y_train, Y_validate = train_test_split(X_relt, Y, test_size=test_size, random_state=seed)

print(f'Y_train Distribution: {Counter(Y_train).most_common()}')

Number of cases: 343
Number of codes (should match): 343
Y_train Distribution: [(0.0, 183), (1.0, 91)]


In [20]:
######################################################
# Oversample to desirable ratio
######################################################

X_balanced, Y_balanced = resample_data(
    X_relt, Y, 
    undersample=undersample, 
    sampling_ratio=sampling_ratio)

Y_train: Counter({0.0: 229, 1.0: 114})
Y_resample: Counter({1.0: 229, 0.0: 229})


In [21]:
######################################################
# Use different algorithms to build models
######################################################

models = []
models.append(('K-Nearest Neighbors (KNN)', KNeighborsClassifier()))
models.append(('Random Forest (RF)', RandomForestClassifier(random_state=seed)))
models.append(('Decision Tree (DT)', DecisionTreeClassifier(random_state=seed)))
models.append(('Multinomial Naive Bayes (MNB)', MultinomialNB()))
models.append(('Logistic Regression (LR)', LogisticRegression(random_state=seed)))
models.append(('Support Vector Machine (SVM)', SVC(gamma='auto')))

# Evaluate algorithms using 10-fold cross validation
results, names = show_kfold_output(models=models, 
                                   X=X_balanced, 
                                   Y=Y_balanced)

K-Nearest Neighbors (KNN):

Mean (std):	 0.6266 (0.4837)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[161  68]
 [ 10 219]]

Report:
               precision    recall  f1-score   support

         0.0       0.94      0.70      0.81       229
         1.0       0.76      0.96      0.85       229

    accuracy                           0.83       458
   macro avg       0.85      0.83      0.83       458
weighted avg       0.85      0.83      0.83       458


Random Forest (RF):

Mean (std):	 0.4716 (0.4992)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[221   8]
 [ 21 208]]

Report:
               precision    recall  f1-score   support

         0.0       0.91      0.97      0.94       229
         1.0       0.96      0.91      0.93       229

    accuracy                           0.94       458
   macro avg       0.94      0.94      0.94       458
weighted avg       0.94      0.94      0.94       458


Decision Tree (DT):

Mean (std):	 0.5175 (0.4997)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [

In [22]:
######################################################
# Save best model
######################################################

rf_relt = RandomForestClassifier(random_state=seed).fit(X_balanced, Y_balanced)
joblib.dump(rf_relt, relt_model_fp)

['/home/jovyan/work/classification/models/classifier_relt_121620.joblib']

## Evaluate algorithms: Demographic perspective

In [23]:
######################################################
# Prepare training and validation data
######################################################

# Separate training and final validation data set. First reltove class
# label from data (X). Setup target class (Y)
# Then make the validation set 10% of the entire
# set of labeled data (X_validate, Y_validate)

demog_df = demog_df[['text', 'demographic_score']]
print("Number of cases:", str(X_demog.shape[0]))

valueArray = demog_df.values
Y = valueArray[:,1]
Y = Y.astype('float')
print("Number of codes (should match):", str(len(Y)))

test_size = 0.2
seed = 3
X_train, X_validate, Y_train, Y_validate = train_test_split(X_demog, Y, test_size=test_size, random_state=seed)

print(f'Y_train Distribution: {Counter(Y_train).most_common()}')

Number of cases: 349
Number of codes (should match): 349
Y_train Distribution: [(0.0, 195), (1.0, 84)]


In [24]:
######################################################
# Oversample to desirable ratio
######################################################

X_balanced, Y_balanced = resample_data(
    X_demog, Y, 
    undersample=undersample, 
    sampling_ratio=sampling_ratio)

Y_train: Counter({0.0: 248, 1.0: 101})
Y_resample: Counter({1.0: 248, 0.0: 248})


In [25]:
######################################################
# Use different algorithms to build models
######################################################

models = []
models.append(('K-Nearest Neighbors (KNN)', KNeighborsClassifier()))
models.append(('Random Forest (RF)', RandomForestClassifier(random_state=seed)))
models.append(('Decision Tree (DT)', DecisionTreeClassifier(random_state=seed)))
models.append(('Multinomial Naive Bayes (MNB)', MultinomialNB()))
models.append(('Logistic Regression (LR)', LogisticRegression(random_state=seed)))
models.append(('Support Vector Machine (SVM)', SVC(gamma='auto')))

# Evaluate algorithms using 10-fold cross validation
results, names = show_kfold_output(models=models, 
                                   X=X_balanced, 
                                   Y=Y_balanced)

K-Nearest Neighbors (KNN):

Mean (std):	 0.6653 (0.4719)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[166  82]
 [  0 248]]

Report:
               precision    recall  f1-score   support

         0.0       1.00      0.67      0.80       248
         1.0       0.75      1.00      0.86       248

    accuracy                           0.83       496
   macro avg       0.88      0.83      0.83       496
weighted avg       0.88      0.83      0.83       496


Random Forest (RF):

Mean (std):	 0.4919 (0.4999)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[244   4]
 [  8 240]]

Report:
               precision    recall  f1-score   support

         0.0       0.97      0.98      0.98       248
         1.0       0.98      0.97      0.98       248

    accuracy                           0.98       496
   macro avg       0.98      0.98      0.98       496
weighted avg       0.98      0.98      0.98       496


Decision Tree (DT):

Mean (std):	 0.5403 (0.4984)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [

In [26]:
######################################################
# Save best model
######################################################

rf_demog = RandomForestClassifier(random_state=seed).fit(X_balanced, Y_balanced)
joblib.dump(rf_demog, demog_model_fp)

['/home/jovyan/work/classification/models/classifier_demog_121620.joblib']